## Conversión de Variables Categóricas

Este cuaderno se basa en el tema 3 del curso de Intermediate Machine Learning de Kaggle. Durante nuestro trabajo con diferentes conjuntos de datos nos encontraemos con muchas variables categóricas, en este cuaderno revisaremos como podemos utilizarlas.

### Introducción

Una variable categórica sólo admite un número limitado de valores.

- Considere una encuesta en la que se pregunta con qué frecuencia desayuna y se ofrecen cuatro opciones: "Nunca", "Rara vez", "La mayoría de los días" o "Todos los días". En este caso, los datos son categóricos, porque las respuestas entran en un conjunto fijo de categorías.
- Si la gente respondiera a una encuesta sobre qué marca de coche tiene, las respuestas entrarían en categorías como "Honda", "Toyota" y "Ford". En este caso, los datos también son categóricos.

Obtendrá un error si intenta introducir estas variables en la mayoría de los modelos de aprendizaje automático en Python sin preprocesarlas primero. En este tutorial, compararemos tres enfoques que puede utilizar para preparar sus datos categóricos.

### Tres enfoques

#### 1. Eliminar las variables categóricas

La forma más sencilla de tratar las variables categóricas es eliminarlas del conjunto de datos. Este enfoque sólo funcionará bien si las columnas no contienen información útil.

#### 2. Codificación ordinal

La codificación ordinal le asigna un valor entero a cada valor único.

![Cofiicación Ordinal](https://storage.googleapis.com/kaggle-media/learn/images/tEogUAr.png)

Este enfoque supone una ordenación de las categorías: "Never" (0) < "Rarely" (1) < "Most days" (2) < "Every day" (3).

Esta suposición tiene sentido en este ejemplo, porque existe una clasificación indiscutible de las categorías. No todas las variables categóricas tienen un orden claro en los valores, pero nos referimos a las que sí lo tienen como variables ordinales. Para los modelos basados en árboles (como los árboles de decisión y los bosques aleatorios), se puede esperar que la codificación ordinal funcione bien con variables ordinales.

#### 3. One-Hot Encoding (Codificación en caliente)

La codificación one-hot crea nuevas columnas que indican la presencia (o ausencia) de cada valor posible en los datos originales. Para entenderlo, vamos a trabajar con un ejemplo.

![one hot](https://storage.googleapis.com/kaggle-media/learn/images/TW5m0aJ.png)

En el conjunto de datos original, "Color" es una variable categórica con tres categorías: "Rojo", "Amarillo" y "Verde". La codificación one-hot correspondiente contiene una columna para cada valor posible y una fila para cada fila del conjunto de datos original. Si el valor original era "Rojo", ponemos un 1 en la columna "Rojo"; si el valor original era "Amarillo", ponemos un 1 en la columna "Amarillo", y así sucesivamente.

A diferencia de la codificación ordinal, la codificación de un punto no presupone un orden de las categorías. Por tanto, puede esperar que este enfoque funcione especialmente bien si no hay un orden claro en los datos categóricos (por ejemplo, "Rojo" no es ni más ni menos que "Amarillo"). Nos referimos a las variables categóricas sin una clasificación intrínseca como variables nominales.

La codificación de un solo valor no suele funcionar bien si la variable categórica toma un gran número de valores (es decir, no se suele utilizar para variables que tomen más de 15 valores diferentes).

### Ejemplo

Trabajaremos con el dataset de información de casas de [Melbourne](https://www.kaggle.com/datasets/dansbecker/melbourne-housing-snapshot).

#### Pero primero cargamos las bibliotecas a utilizar.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


#### Cargamos el dataset

In [2]:
# Colocamos la ruta al conjunto de datos en una variable para facilitar el acceso
melbourne_file_path = "https://raw.githubusercontent.com/vbatiz/intro-ML/main/notebooks/datasets/melb_data.csv"
# Leemos los datos y los almacenamos en un dataframe llamado melbourne_data
melbourne_data = pd.read_csv(melbourne_file_path)
# Desplegamos un resumen de los datos del dataframe
melbourne_data.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [3]:
melbourne_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

In [4]:
melbourne_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


#### Separamos las variables en objetivo y predictoras  

In [5]:
y = melbourne_data.Price
X = melbourne_data.drop(['Price'], axis=1)

#### Dividimos los datos en entrenamiento y validación

In [6]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

#### Eliminamos las columnas con valores nulos (enfoque más sencillo)

In [7]:
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] #Obtenemos las columnas con valores nulos
X_train_full.drop(cols_with_missing, axis=1, inplace=True) # Usando inplace no ocupamos reasiganr la salida a la variable
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

In [8]:
X_train_full.shape

(10864, 16)

#### Seleccionar columnas categóricas con cardinalidad relativamente baja (conveniente pero arbitrario)

"Cardinalidad": número de valores únicos de una columna.

In [9]:
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and
                        X_train_full[cname].dtype == "object"]

In [11]:
print(low_cardinality_cols)

['Type', 'Method', 'Regionname']


#### Seleccionar las columnas numéricas

In [12]:
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [13]:
print(numerical_cols)

['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude', 'Propertycount']


#### Nos quedamos solo con las columnas seleccionadas

In [14]:
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [15]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


A continuación, obtenemos una lista de todas las variables categóricas de los datos de entrenamiento.

Para ello, comprobamos el tipo de datos (o dtype) de cada columna. El objeto dtype indica que una columna tiene texto (teóricamente podría ser otras cosas, pero eso carece de importancia para nuestros fines). Para este conjunto de datos, las columnas con texto indican variables categóricas.

In [16]:
# Obtenemos la lista de las variables actegóricas
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Variables Categóricas:")
print(object_cols)

Variables Categóricas:
['Type', 'Method', 'Regionname']


#### Definir una función para medir la calidad de cada enfoque

Definimos una función score_dataset() para comparar los tres enfoques diferentes para tratar variables categóricas. Esta función informa del error medio absoluto (MAE) de un modelo de bosque aleatorio. En general, queremos que el MAE sea lo más bajo posible.

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# FunctionFunción para comparar los diferentes enfoques
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

#### Puntuación del método 1 (eliminar variables categóricas)
Eliminamos las columnas de objetos con el método select_dtypes().

In [18]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

In [19]:
drop_X_train.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


In [20]:
print("MAE para el enfoque 1 (Eliminar las variables categoricas):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE para el enfoque 1 (Eliminar las variables categoricas):
175703.48185157913


#### Puntuación del enfoque 2 (codificación ordinal)
Scikit-learn tiene una clase OrdinalEncoder que se puede utilizar para obtener codificaciones ordinales. Hacemos un ciclo sobre las variables categóricas y aplicamos el codificador ordinal por separado a cada columna.

In [21]:
from sklearn.preprocessing import OrdinalEncoder

# Hacemos una copia para evitar cambiar los datos originales
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

In [22]:
label_X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


In [23]:
# Applicamos ordinal encoder a cada columna con datos categóricos
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

In [26]:
label_X_valid.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
8505,0.0,3.0,6.0,4,8.0,3016.0,4.0,2.0,450.0,-37.86100,144.89850,6380.0
5523,0.0,1.0,6.0,2,6.6,3011.0,2.0,1.0,172.0,-37.81000,144.88960,2417.0
12852,0.0,3.0,6.0,3,10.5,3020.0,3.0,1.0,581.0,-37.76740,144.82421,4217.0
4818,1.0,0.0,5.0,3,4.5,3181.0,2.0,2.0,128.0,-37.85260,145.00710,7717.0
12812,0.0,1.0,2.0,3,8.5,3044.0,3.0,2.0,480.0,-37.72523,144.94567,7485.0


In [25]:
print("MAE para el enfoque 2 (Ordinal Encoding):")
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE para el enfoque 2 (Ordinal Encoding):
165936.40548390493


En la celda de código anterior, para cada columna, asignamos aleatoriamente cada valor único a un número entero diferente. Este es un enfoque común que es más simple que proporcionar etiquetas personalizadas; sin embargo, podemos esperar un aumento adicional en el rendimiento si proporcionamos etiquetas mejor informadas para todas las variables ordinales.

#### Puntuación del enfoque 3 (codificación one-hot)
Utilizamos la clase OneHotEncoder de scikit-learn para obtener codificaciones one-hot. Hay una serie de parámetros que se pueden utilizar para personalizar su comportamiento.

- Establecemos handle_unknown='ignore' para evitar errores cuando los datos de validación contienen clases que no están representadas en los datos de entrenamiento.

Para utilizar el codificador, sólo tenemos que proporcionar las columnas categóricas que queremos que se codifiquen de una en una. Por ejemplo, para codificar los datos de entrenamiento, proporcionamos X_train[object_cols]. (object_cols en la celda de código de abajo es una lista de los nombres de las columnas con datos categóricos, por lo que X_train[object_cols] contiene todos los datos categóricos del conjunto de entrenamiento).

In [29]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Aplicamos one-hot encoder a cada columna con datos categóricos

#OH_encoder = ColumnTransformer([("One_hot_encoder", OneHotEncoder(sparse=False, categories='auto'))], remainder='passthrough')

#OH_encoder = OneHotEncoder(handle_unknown='ignore')
#OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
#OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

OH_cols_train = pd.get_dummies(X_train, columns=object_cols)
OH_cols_valid = pd.get_dummies(X_valid, columns=object_cols)


In [30]:
OH_cols_train.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount,Type_h,...,Method_SP,Method_VB,Regionname_Eastern Metropolitan,Regionname_Eastern Victoria,Regionname_Northern Metropolitan,Regionname_Northern Victoria,Regionname_South-Eastern Metropolitan,Regionname_Southern Metropolitan,Regionname_Western Metropolitan,Regionname_Western Victoria
12167,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0,False,...,False,False,False,False,False,False,False,True,False,False
6524,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0,True,...,False,False,False,False,False,False,False,False,True,False
8413,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0,True,...,False,False,False,False,False,False,False,False,True,False
2919,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0,False,...,True,False,False,False,True,False,False,False,False,False
6043,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0,True,...,False,False,False,False,False,False,False,False,True,False


In [31]:
OH_cols_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10864 entries, 12167 to 2732
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Rooms                                  10864 non-null  int64  
 1   Distance                               10864 non-null  float64
 2   Postcode                               10864 non-null  float64
 3   Bedroom2                               10864 non-null  float64
 4   Bathroom                               10864 non-null  float64
 5   Landsize                               10864 non-null  float64
 6   Lattitude                              10864 non-null  float64
 7   Longtitude                             10864 non-null  float64
 8   Propertycount                          10864 non-null  float64
 9   Type_h                                 10864 non-null  bool   
 10  Type_t                                 10864 non-null  bool   
 11  Type

In [ ]:
# One-hot encoding elimina el index; lo regresamos
#OH_cols_train.index = X_train.index
#OH_cols_valid.index = X_valid.index
# Removemos las columnas categóricas (serán remplazadas por las columnas one-hot encoding)
#num_X_train = X_train.drop(object_cols, axis=1)
#num_X_valid = X_valid.drop(object_cols, axis=1)
# Agregamos las columnas one-hot encoded a las columnas numéricas
#OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
#OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)
# Nos aseguramos que todas las columnas tienen el tipo String
#OH_X_train.columns = OH_X_train.columns.astype(str)
#OH_X_valid.columns = OH_X_valid.columns.astype(str)

In [ ]:
OH_cols_train.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount,Type_h,...,Method_SP,Method_VB,Regionname_Eastern Metropolitan,Regionname_Eastern Victoria,Regionname_Northern Metropolitan,Regionname_Northern Victoria,Regionname_South-Eastern Metropolitan,Regionname_Southern Metropolitan,Regionname_Western Metropolitan,Regionname_Western Victoria
12167,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0,False,...,False,False,False,False,False,False,False,True,False,False
6524,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0,True,...,False,False,False,False,False,False,False,False,True,False
8413,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0,True,...,False,False,False,False,False,False,False,False,True,False
2919,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0,False,...,True,False,False,False,True,False,False,False,False,False
6043,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0,True,...,False,False,False,False,False,False,False,False,True,False


In [ ]:
print(object_cols)

['Type', 'Method', 'Regionname']


In [32]:
print("MAE para el enfoque 3 (One-Hot Encoding):")
print(score_dataset(OH_cols_train, OH_cols_valid, y_train, y_valid))

MAE para el enfoque 3 (One-Hot Encoding):
166089.4893009678


#### ¿Qué método es el mejor?
En este caso, la eliminación de las columnas categóricas (método 1) fue la que obtuvo peores resultados, ya que obtuvo la puntuación MAE más alta. En cuanto a los otros dos enfoques, como las puntuaciones MAE obtenidas están tan próximas, no parece que haya ninguna ventaja significativa entre uno y otro.

En general, la codificación de una sola vez (método 3) suele dar mejores resultados y la eliminación de las columnas categóricas (método 1) suele dar peores resultados, pero varía en función de cada caso.

#### Conclusión
El mundo está lleno de datos categóricos. Serás un científico de datos mucho más eficaz si sabes cómo utilizar este tipo de datos tan común.

### Ejemplo usando el dataset del Titanic

El dataset lo tenemos disponible en el repositorio de Intro a Python.

In [68]:
# Colocamos la ruta al conjunto de datos en una variable para facilitar el acceso
titanic_file_path = "https://raw.githubusercontent.com/vbatiz/intro-python/main/notebooks/data/titanic.csv"
# Leemos los datos y los almacenamos en un dataframe llamado melbourne_data
df_titanic = pd.read_csv(titanic_file_path)
# Desplegamos un resumen de los datos del dataframe
df_titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [69]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [70]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

### Seleccionamos los atributos que usaremos como predictores y la variable a predecir (sobrevivencia)

In [86]:
#columnas = ['Pclass','Sex','Age','SibSp','Fare']
columnas = ['Pclass','Sex','Age','Fare']
X = df_titanic[columnas]
y = df_titanic['Survived']

In [87]:
X.head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


In [88]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

### Convertimos las variables categóricas a su representación numérica

In [89]:
columnas_categoricas = ['Sex']
X = pd.get_dummies(X, columns=columnas_categoricas)

In [90]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Age         714 non-null    float64
 2   Fare        891 non-null    float64
 3   Sex_female  891 non-null    bool   
 4   Sex_male    891 non-null    bool   
dtypes: bool(2), float64(2), int64(1)
memory usage: 22.7 KB


### Colocamos la edad promedio a los registros que tengan valor nulo en la edad

In [91]:
X.Age = X.Age.fillna(X.Age.mean())

### Creamos los conjuntos de datos de entrenamiento y validación

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Revisamos la mejor opción para el modelo

In [93]:
modelo_knn = KNeighborsClassifier()
modelo_knn.fit(X_train, y_train)

KNeighborsClassifier()

In [94]:
# Evaluamos de forma general el modelo
modelo_knn.score(X_test, y_test)

0.7318435754189944

In [95]:
y_test_pred = modelo_knn.predict(X_test)

In [96]:
f1_valor = f1_score(y_test, y_test_pred)

In [97]:
print(f1_valor)

0.6250000000000001


In [98]:
accuracy_valor = accuracy_score(y_test, y_test_pred)

In [99]:
print(accuracy_valor)

0.7318435754189944
